In [1]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [2]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [3]:
# Environment Variables

# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')

# Gemini Environment Variables
gemini_api_key = env_to_str("GEMINI_API_KEY")
gemini_base_url = env_to_str("GEMINI_BASE_URL")

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [4]:
# Available AI Models

# Ollama General Models
model_mistral = env_to_str("MODEL_MISTRAL")
model_llama = env_to_str('MODEL_LLAMA3')
model_mistral_nemo = env_to_str("MODEL_MISTRAL_NEMO")
model_gemma = env_to_str("MODEL_GEMMA3")

# Ollama Resoning Models 
model_phi = env_to_str("MODEL_PHI3.5")
model_qwen = env_to_str("MODEL_QWEN3")
model_deepseek = env_to_str("MODEL_DEEPSEEK_R1")

# Gemini Models
model_gemini_flash = env_to_str("MODEL_GEMINI_FLASH")

In [ ]:
# Configurations

# Set Current Model
current_model = model_llama

# Multi Model Flag
multi_model = False

# Check for the need of multiple models, 
# and if not, make sure to have selected 
# a current ai model.
if multi_model and current_model != "":
    print("This is a multi model setup. Leave current model as empty string: ''.")
elif multi_model and current_model == "":
    print("This is a multi model setup. ")
elif not multi_model and current_model == "":
    print("This is a single model setup. Please set a current model.")
elif not multi_model and current_model != "":
    print(f"Not a multi model setup. The current model is: {model_llama}")

Not a multi model setup. The current model is: llama3.1:8b


In [6]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

instructions4 = "You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only."

In [7]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [8]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_llama,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [9]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=ollama_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=ollama_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=ollama_model
)

sales_picker = Agent(
    name="sales_picker",
    instructions=instructions4,
    model=ollama_model
)

In [10]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name) as current_span:
        if agents:
            
            # Trace Attributes
            current_span.set_attribute("user.request", message)
            current_span.set_attribute("model.name", model_llama)
            
            for index, agent in enumerate(agents):
                
                # Trace Attributes
                current_span.set_attribute("agent.name", agent.name)
        
                results = await asyncio.gather(
                    *[Runner.run(agent, message) for agent in agents]
                )

                outputs = []

                for result in results:
                    outputs.append(result.final_output)

                combined_outputs = "\n\n--- Option ---\n\n".join(outputs)
                best = await Runner.run(agents[-1], combined_outputs)

                if outputs:
                    total_length = sum(len(output) for output in outputs)
                    
                    # Add response attributes
                    current_span.set_attribute("response.length", total_length)
                    # current_span.set_attribute("response.preview", outputs[0][:100] if outputs[0] else "")
                    current_span.set_attribute("response.preview", outputs[0] if outputs[0] else "")
        else:
            print("No additional arguments provided.")

        # outputs = [result.final_output for result in results]
        
        
        for output in outputs:
            display(Markdown(f"# Sales Agent:\n\n{output}\n\n"))
            display(Markdown(f"# Sales Picker:\n\n{best}"))
        return outputs, best

In [ ]:
# Result
result, best = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3,
    sales_picker
)

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/opentelemetry/trace/__init__.py", line 589, in use_span
    yield span
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/openinference/instrumentation/_tracers.py", line 141, in start_as_current_span
    yield cast(OpenInferenceSpan, current_span)
  File "/var/folders/mz/sc9xd4w571bctk3y250vd0dm0000gn/T/ipykernel_5970/2744680573.py", line 16, in phoenix_tracer
    results = await asyncio.gather(
              ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.

CancelledError: 

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
